In [29]:
import numpy as np
import pandas as pd
import glob
import os
from pathlib import Path
from itertools import product

DATA = '/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data'

#right, left, jump, fire
REMAP = {'Key.right': 'R', 
         'Key.left': 'L', 
         '\'d\'': 'J', 
         'Key.space': 'J',
         'Key.up': 'J'}

FPS = 60
N_FRAME = [2*FPS, 10*FPS, 60*FPS, 300*FPS]
BLOCK_TIME = ['2s', '10s', '1min', '5min']
MIN_OCCUR = [1, 2, 5, 20]
p3 = [''.join(p) for n in range(2,4) for p in product('RLJ', repeat=n)]
p4 = [''.join(p) for n in range(2,5) for p in product('RLJ', repeat=n)]
p10 = [''.join(p) for n in range(2,11) for p in product('RLJ', repeat=n)]
p15 = [''.join(p) for n in range(2,16) for p in product('RLJ', repeat=n)]
PATTERN_MAX_LENGTH = [3, 4, 10, 15]
PATTERNS = [p3, p4, p10, p15]


def slice_keylog_file(frames_and_inputs_list, predicted=True):

    for i, run in enumerate(frames_and_inputs_list):
        for block_time, block_size in zip(BLOCK_TIME, N_FRAME):
            block = list()
            n_block = int(run.FRAME.max()/block_size)

            for period in range(n_block):
                slice = run[(run.FRAME >= period*block_size) & (run.FRAME < (period+1)*block_size)]
                block.append(slice)

                if predicted:
                    save_path = DATA + '/sliced_logs/predicted/run_{}/{}/slice{}.csv'.format(i, block_time, period) 
                else:
                    save_path = DATA + '/sliced_logs/ground_truth/run_{}/{}/slice{}.csv'.format(i, block_time, period) 

                filepath2 = Path(save_path)  
                filepath2.parent.mkdir(parents=True, exist_ok=True)  
                slice.to_csv(filepath2) 


#read csv file (inputs), return list of pressed keys (down only and standardized as LRJ)
def read_and_standardize_logs(csv_file, predicted=True):

    data = pd.read_csv(csv_file, index_col=0)
    #data['run_number'] = i

    main_keys = data.KEY.value_counts().keys()[:4].tolist() #might not be necessary for predicted inputs
    data =  data[data.KEY.isin(main_keys)]


    keylogs = data[(data.KEY != '\'a\'') & (data.KEY != 'Key.enter') & (data.KEY != '\'s\'')] # A modifier quand on saura ce que predit exactement le modele
    keylogs = keylogs.replace({"KEY": REMAP}) #idem
    keylogs = keylogs[keylogs.STATUS == 'DOWN'] #on garde que les touches pressées atm


    #slice_keylog_file(frames_and_inputs_list, predicted)

    return keylogs



#inputs_log : must contain a "KEY" column with only letter L,R,J
#return Series of pattern count in decreasing order; the pattern is the index
def get_pattern_count(standardized_inputs, max_pattern_size, min_pattern_occur=10):
    patterns = [''.join(p) for n in range(2,max_pattern_size+1) for p in product('RLJ', repeat=n)]
    
    d=dict()
    keys_as_str = standardized_inputs.KEY.str.cat()
    
    for pattern in patterns:
        d[pattern]=keys_as_str.count(pattern)

    frequencies = pd.Series(dict(sorted(d.items(), key=lambda x: x[1], reverse=True)), name='pattern_count')
    
    return frequencies[frequencies >= min_pattern_occur]

In [ ]:
base_path = '/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/test/sliced_logs/ground_truth'
for i in range(23):
    path1 = base_path + '/run_{}'.format(i)
    
    for length in BLOCK_TIME:
        path_in = path1 + '/' + length

        if os.path.exists(path1):
            CSVs = sorted(Path(path_in).glob('*.csv'))

            for j,f in enumerate(CSVs):
                series = get_pattern_count(read_and_standardize_logs(f, False), 2, )

In [30]:
run1_2s_s11 = '/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/test/sliced_logs/ground_truth/run_1/2s/slice11.csv'
run1_10s_s89 = '/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/test/sliced_logs/ground_truth/run_1/10s/slice89.csv'
run1_1m_s14 = '/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/test/sliced_logs/ground_truth/run_1/1min/slice14.csv'
run1_5m_s3 = '/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/test/sliced_logs/ground_truth/run_1/5min/slice3.csv'

f1 = get_pattern_count(read_and_standardize_logs(run1_2s_s11, False), 3, 1)
f2 = get_pattern_count(read_and_standardize_logs(run1_10s_s89, False), 4, 2)
f3 = get_pattern_count(read_and_standardize_logs(run1_1m_s14, False), 8, 5)
f4 = get_pattern_count(read_and_standardize_logs(run1_5m_s3, False), 10, 10)

f = [f1, f2, f3, f4]

for i, item in enumerate(f):
    item.to_csv('/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/test/f_{}.csv'.format(i))

In [33]:
pd.read_csv(run1_5m_s3, index_col=0)

,FRAME,KEY,STATUS,run_number
2796,54061,J,DOWN,1
2797,54068,L,DOWN,1
2800,54117,J,DOWN,1
2802,54150,L,DOWN,1
2805,54171,R,DOWN,1
...,...,...,...,...
3409,71343,J,DOWN,1
3412,71441,R,DOWN,1
3414,71815,R,DOWN,1
3416,71948,J,DOWN,1
